In [1]:
%cd ..

/home/ead/iking5/code/StreamLP


In [52]:
import torch 
import matplotlib.pyplot as plt 
from torch_geometric.nn import MessagePassing 
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, precision_score, recall_score
from math import ceil

In [62]:
def test_nonparam_lp(d, aggr='mean', dataset='digg'):
    g = torch.load(f'StrGNN_Data/{dataset}.pt')

    rnd_x = torch.eye(d).repeat(
        ceil(g.num_nodes / d), 1
    )[:g.num_nodes]
    rnd_x = rnd_x[torch.randperm(rnd_x.size(0))]
    
    #rnd_x = torch.rand(g.num_nodes,d)

    ei = g.edge_index[:, g.edge_index.size(1)//2:]
    te_ei = g.edge_index[:, :g.edge_index.size(1)//2]

    mp = MessagePassing(aggr=aggr)
    one_hop = torch.cat([
        mp.propagate(g.edge_index, size=None, x=rnd_x),
        mp.propagate(g.edge_index[[1,0]], size=None, x=rnd_x),
    ], dim=1)
    two_hops = torch.cat([
        mp.propagate(g.edge_index, size=None, x=one_hop),
        mp.propagate(g.edge_index[[1,0]], size=None, x=one_hop),
    ], dim=1)
    three_hops = torch.cat([
        mp.propagate(g.edge_index, size=None, x=two_hops),
        mp.propagate(g.edge_index[[1,0]], size=None, x=two_hops),
    ], dim=1)

    x = torch.cat([one_hop,two_hops,three_hops], dim=1)
    edges = torch.cat([x[ei[0]], x[ei[1]]], dim=1)
    fake = torch.cat([
        x[torch.randint(0, g.num_nodes, (edges.size(0),))], 
        x[torch.randint(0, g.num_nodes, (edges.size(0),))]
    ], dim=1)

    y = torch.ones(edges.size(0)*2)
    y[:y.size(0)//2] = 0
    rf = RandomForestClassifier(n_jobs=16)
    rf.fit(torch.cat([edges, fake], dim=0), y)

    te_edges = torch.cat([x[te_ei[0]], x[te_ei[1]]], dim=1)
    te_fake = torch.cat([
        x[torch.randint(0, g.num_nodes, (te_edges.size(0),))], 
        x[torch.randint(0, g.num_nodes, (te_edges.size(0),))]
    ], dim=1)

    y_hat = rf.predict(torch.cat([te_edges, te_fake], dim=0))
    labels = torch.ones(te_edges.size(0)*2)
    labels[:labels.size(0)//2] = 0 

    print(
        accuracy_score(labels, y_hat),
        precision_score(labels, y_hat),
        recall_score(labels, y_hat),
        sep='\t'
    )


for a in ['mean', 'sum', 'max']:
    print(a)
    print("d\tAccuracy\t\tPrecision\t\tRecall")
    for d in [2,4,8,16]:
        print(str(d)+'\t', end='')
        test_nonparam_lp(d, aggr=a)

    print()

mean
d	Accuracy		Precision		Recall
2	0.9508365097117294	0.9899055036087204	0.9109624997146966
4	0.9587451213110264	0.9913219908086438	0.9255928605664985
8	0.9507223883322301	0.9843516224767604	0.9160066646885627
16	0.9417182114897404	0.9685502614758861	0.9130851573733824

sum
d	Accuracy		Precision		Recall
2	0.9400748636249515	0.9685540704738761	0.9096843402643051
4	0.9323716705087531	0.9585582532497398	0.9038185013580444
8	0.925524387738799	0.9487543627391984	0.8996416588683724
16	

In [51]:
g.edge_index[:,:25]

tensor([[  0,  50,  90, 123,  33, 151, 116, 128, 175, 175, 151, 189, 151, 230,
         244, 246, 268, 324, 226,  50, 360, 391, 388, 196, 309],
        [  1,  51,  91, 124,  34, 152, 117, 163, 176, 151, 233, 190, 267, 231,
         245, 245, 267, 325, 151, 360, 360, 392, 389, 360, 310]])